# Neuralmt program

In [11]:
from default import *
import os, sys

## Run the solution on dev

In [12]:
model = Seq2Seq(build=False)
model.load(os.path.join('data', 'seq2seq_E049.pt'))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()
# loading test dataset
test_iter = loadTestData(os.path.join('data', 'input', 'dev.txt'), model.fields['src'],
                            device=device, linesToLoad=sys.maxsize)
results = translate(model, test_iter) # Warning: will take >5mins depending on your machine
print("\n".join(results))

AttributeError: 'Seq2Seq' object has no attribute 'fields'

## Evaluate the default output

In [9]:
from bleu_check import bleu
ref_t = []
with open(os.path.join('data','reference','dev.out')) as r:
    ref_t = r.read().strip().splitlines()
print(bleu(ref_t, results))

ModuleNotFoundError: No module named 'bleu_check'


## Overview
This program uses 2 Gated Recurrent Units (GRU) in a sequence to sequence model with an additional attention mdoel to translate German into English using python 3.8+.

### Components

### 1. AttentionModule
calculates the attention weights 

#### calcAlpha(self, decoder_hidden, encoder_out):
Computes the attention score using the encoders output and the decoders current hidden state. The attention score is calculated by applying the hyperbolic tangent fucntion (tanh) on the encoders output and decoders hidden state, performs a matrix transformation then applys the softmax function to calculate the alpha.
#### forward(self, decoder_hidden, encoder_out):
Computes and returns the context vector and alpha using the calcAlpha function.
The context vector is calculated by multipying alpha from calcAlpha with the encoders hidden state and summing up over all index i.

### 2. def greedyDecoder(decoder, encoder_out, encoder_hidden, maxLen)
Translates the output using a greedy algorithm to select the highest probabilty token and implements unknown token replacement.

### 3. def translate(model, input_dl):
Uses the model to translate the input sequence to the desired output sequence. In this program it converts a German sequence input into an English output.

### 4. hp
Defines all the hyper parameters used in the encoder, decoder and the dataset classes 

Parameters:

-   pad_idx: padding index 
-   sos_idx: start of sequence index
-  eos_idx: end of sequence index
-  unk_idx: unknown token index
-  lex_min_freq = minimum frequency for the lexicon


-   hidden_dim = hidden dimension
-   embed_dim = embedding dimension
-   n_layers = layers
-   dropout = dropout rate
-   batch_size = batch size
-   num_epochs = epoch num
-   lexicon_cap = lexicon capacity

-   cycle_length: length of the cycle

-   max_len: max length for the sequence to sequence model
-   device: cude or CPU

### 5. Encoder
Uses a bidiretional GRU to contextually encode an input sequence. 

### 6. Decoder
Uses a single directional GRU and the attention module to decode the output from the the encoder

### 7. Seq2Seq
Combines the Encoder and Decoder to create a single sequence to sequence model.

### 8. Dataset
A utility class that helps to manage the source data, target data and the tokenizers for German and English








## Analysis

### Attention Module:

The program in its initial state started with a BLEU score of 1.8637.

The calcAlpha function first calculates the score using the encoders output and the decoder current hidden state. Then applys the tanh function on this score. This is finally fed into a softmax function to calculate the alpha.

We initially faced dimension issues with running calcAlpha. We were able to fix these issues by squeezing the scores before the softmax function. 

The forward function calculates the context vector using the sum of the output of calcAlpha multiplied by the decoders hidden state.

Running on a subset of dev we were able to get a BLEU score of 12.0102
Running on the full test set  we were able to get a BLEU score of 14.2427



### Translate Function:

Our translate function attempts to implements the unknown word replacement but was unable to find unk words in the decoders output.

### Observations:

- Implementing an attention module greatly increases the performance of a NMT model.
- The attention module does not require as much work as expected to create a much better translation

## Conclusion:
- What we learned
- - For NMT using a sequence to sequence model, it is necessary to include an attention module to greatly increse the quality and accuracy of translations.
- Future Works 
- - Fixing our unknown word replacement to see how much of an improvement it adds to the translation
- - Implementing beam search to compared the performance gains to just unk replacement and combined with unk replacement






